In [136]:
import praw
import pandas as pd

In [137]:
reddit = praw.Reddit(
    client_id='IDPMLFiJcLQO0LSX2_iokw',
    client_secret='YnB6wbk5MdfnVsc4gI5Pc3Aaj1GBLQ',
    user_agent='assignment',
)

In [138]:
reddit.read_only

True

# Collecting post and comments from r/python

In [139]:
posts = reddit.subreddit('python').top(time_filter='week',limit=10)

In [159]:
from praw.models import MoreComments

post_list, comment_list = [],[]

def fetch_comments(comments_obj,post_id):
    
    q = []
    for i,comment in enumerate(comments_obj):
        if i>20:
            break
        if isinstance(comment,MoreComments):
            continue
            
        q.append((comment,0))
    
    while(q):
        comment,lvl = q.pop()
        
        comment_list.append({
            'post_id':post_id,
            'comment_id':comment.id,
            'parent_id':comment.parent_id,
            'body':comment.body,
            'created_on':comment.created_utc,
            'upvotes':comment.score,
            'author_id':comment.author.id if comment.author and hasattr(comment.author,'id') else 'deleted',
            'author_name':comment.author.name if comment.author and hasattr(comment.author,'name') else 'deleted',
        })
        
        if lvl > 0:
            continue
        
        for i,neigh_comment in enumerate(comment.replies):
            if isinstance(neigh_comment,MoreComments):
                continue
            if i >10:
                break
            q.append((neigh_comment,lvl+1))
            
        
            
        
def fetch_posts(subreddit,post_types = ['rising','hot','top week'],limit=10):
    visited = set()
    for post_type in post_types:
        if len(post_type.split(' '))==2 and post_type.split(' ')[0]=='top':
            posts = reddit.subreddit(subreddit).top(time_filter=post_type.split(' ')[1],limit=limit)
        elif post_type =='top':
            posts = reddit.subreddit(subreddit).top(time_filter='week',limit=limit)
        elif post_type=='rising':
            posts = reddit.subreddit(subreddit).rising(limit=limit)
        elif post_type=='hot':
            posts = reddit.subreddit(subreddit).hot(limit=limit)
            
        for post in posts:
            if post.id in visited:
                continue
            visited.add(post.id)
            post_list.append({
                'id':post.id,
                'author':post.author.name if post.author and hasattr(post.author,'name') else 'deleted',
                'author_id':post.author.id if post.author and hasattr(post.author,'id') else 'deleted',
                'total_comments':post.num_comments,
                'upvote':post.score,
                'post_type':post_type,
                'title':post.title,
                'body':post.selftext,
                'downvote':post.score*(1-post.upvote_ratio)/post.upvote_ratio,
                'url': post.url,
                'created_on':post.created_utc,
                'subreddit':subreddit,
                'subreddit_id':post.subreddit_id
            })
            
            fetch_comments(post.comments,post.id)

In [160]:
fetch_posts('python',['top week'])

In [161]:
post_list

[{'id': '12glkw4',
  'author': 'TheBodyPolitic1',
  'author_id': 'von3w6y2',
  'total_comments': 319,
  'upvote': 591,
  'post_type': 'top week',
  'title': "Why didn't Python become popular until long after its creation?",
  'body': "Python was invented in 1994, two years before Java.\n\nGiven it's age, why didn't Python become popular or even widely known about, until much later?",
  'downvote': 37.72340425531918,
  'url': 'https://www.reddit.com/r/Python/comments/12glkw4/why_didnt_python_become_popular_until_long_after/',
  'created_on': 1681051909.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': '12hj9oc',
  'author': 'MetonymyQT',
  'author_id': 'shnqm',
  'total_comments': 80,
  'upvote': 527,
  'post_type': 'top week',
  'title': 'Free course: Build a modern API with FastAPI and Python',
  'body': "Hello everyone! \n\nI've posted this course 4 months ago on this sub Reddit and it was well received. I want to do another giveaway,\n\nAll 3 coupons expire in 4 day

In [162]:
comment_list

[{'post_id': '12glkw4',
  'comment_id': 'jfkvwx0',
  'parent_id': 't3_12glkw4',
  'body': "Hardware wasn't ready for Python in that time",
  'created_on': 1681054038.0,
  'upvotes': 5,
  'author_id': '22vat21u',
  'author_name': 'DusikOff'},
 {'post_id': '12glkw4',
  'comment_id': 'jfnha98',
  'parent_id': 't3_12glkw4',
  'body': 'Because Python was developed with the conceit that memory is cheap and plentiful. It took time for consumer level hardware make this true.\n\n[Massive memory overhead: Numbers in Python and how NumPy helps](https://pythonspeed.com/articles/python-integers-memory/)',
  'created_on': 1681095316.0,
  'upvotes': 2,
  'author_id': '4wtjvsh6',
  'author_name': 'FredVIII-DFH'},
 {'post_id': '12glkw4',
  'comment_id': 'jflnf2e',
  'parent_id': 't3_12glkw4',
  'body': 'Perl was *the* scripting language in the early-mid 2000s. I took a look at it and hated the syntax. Python made much more sense to me and a lot of other people but it took time for a critical mass of us

In [163]:
from tinydb import TinyDB

post_db, comment_db = TinyDB('post.json'), TinyDB('comment.json')

In [164]:
post_db.insert_multiple(post_list)
comment_db.insert_multiple(comment_list)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [165]:
post_db.all()

[{'id': '12glkw4',
  'author': 'TheBodyPolitic1',
  'author_id': 'von3w6y2',
  'total_comments': 319,
  'upvote': 591,
  'post_type': 'top week',
  'title': "Why didn't Python become popular until long after its creation?",
  'body': "Python was invented in 1994, two years before Java.\n\nGiven it's age, why didn't Python become popular or even widely known about, until much later?",
  'downvote': 37.72340425531918,
  'url': 'https://www.reddit.com/r/Python/comments/12glkw4/why_didnt_python_become_popular_until_long_after/',
  'created_on': 1681051909.0,
  'subreddit': 'python',
  'subreddit_id': 't5_2qh0y'},
 {'id': '12hj9oc',
  'author': 'MetonymyQT',
  'author_id': 'shnqm',
  'total_comments': 80,
  'upvote': 527,
  'post_type': 'top week',
  'title': 'Free course: Build a modern API with FastAPI and Python',
  'body': "Hello everyone! \n\nI've posted this course 4 months ago on this sub Reddit and it was well received. I want to do another giveaway,\n\nAll 3 coupons expire in 4 day

In [166]:
comment_db.all()

[{'post_id': '12glkw4',
  'comment_id': 'jfkvwx0',
  'parent_id': 't3_12glkw4',
  'body': "Hardware wasn't ready for Python in that time",
  'created_on': 1681054038.0,
  'upvotes': 5,
  'author_id': '22vat21u',
  'author_name': 'DusikOff'},
 {'post_id': '12glkw4',
  'comment_id': 'jfnha98',
  'parent_id': 't3_12glkw4',
  'body': 'Because Python was developed with the conceit that memory is cheap and plentiful. It took time for consumer level hardware make this true.\n\n[Massive memory overhead: Numbers in Python and how NumPy helps](https://pythonspeed.com/articles/python-integers-memory/)',
  'created_on': 1681095316.0,
  'upvotes': 2,
  'author_id': '4wtjvsh6',
  'author_name': 'FredVIII-DFH'},
 {'post_id': '12glkw4',
  'comment_id': 'jflnf2e',
  'parent_id': 't3_12glkw4',
  'body': 'Perl was *the* scripting language in the early-mid 2000s. I took a look at it and hated the syntax. Python made much more sense to me and a lot of other people but it took time for a critical mass of us